FRP

- mean, median, std, min, max for frp values

- total observations

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the dataset
df = pd.read_csv(r"E:\IPMA\FRP\FRP_2001_2023.csv")

# Display the first few rows of the dataset to check the structure
df.head()

# Convert the 'acq_date' column to datetime format
df['acq_date'] = pd.to_datetime(df['acq_date'])

# Extract the year, month, and day from the 'acq_date' column
df['year'] = df['acq_date'].dt.year
df['month'] = df['acq_date'].dt.month
df['day'] = df['acq_date'].dt.day

# Display the dataset to ensure the new columns were added correctly
df.head()


In [ ]:
# Calculate the summary statistics (mean, median, std, min, max) for FRP by year
yearly_stats = df.groupby('year')['frp'].agg(['mean', 'median', 'std', 'min', 'max']).reset_index()

# Display the yearly statistics table
yearly_stats


In [ ]:
# Count the number of observations per year
yearly_counts = df.groupby('year').size().reset_index(name='count')

# Display the table of observations per year
yearly_counts


In [ ]:
# Count the number of observations per month (group by year and month)
monthly_counts = df.groupby(['year', 'month']).size().reset_index(name='count')

# Display the table of observations per month
monthly_counts


In [ ]:
# Count the number of observations per day (group by year, month, and day)
daily_counts = df.groupby(['year', 'month', 'day']).size().reset_index(name='count')

# Display the table of observations per day
daily_counts


In [ ]:
# Plot the mean FRP values per year
plt.figure(figsize=(5, 3))
plt.plot(yearly_stats['year'], yearly_stats['mean'], marker='o', label='Mean FRP')
plt.fill_between(yearly_stats['year'], yearly_stats['mean'] - yearly_stats['std'], yearly_stats['mean'] + yearly_stats['std'], alpha=0.2, label='Standard Deviation')
plt.xlabel('Year')
plt.ylabel('FRP (Mean)')
plt.title('Mean Fire Radiative Power (FRP) by Year')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Plotting the yearly mean FRP to visualize trends
plt.figure(figsize=(5, 3))
plt.plot(yearly_stats['year'], yearly_stats['mean'], marker='o')
plt.title("Trend of Mean FRP Over Time")
plt.xlabel("Year")
plt.ylabel("Mean FRP")
plt.grid(True)
plt.show()


In [ ]:
# Calculate the total FRP per year
total_frp = df.groupby('year')['frp'].sum().reset_index()

# Plot the total FRP by year
plt.figure(figsize=(5, 3))
plt.bar(total_frp['year'], total_frp['frp'], color='skyblue')
plt.xlabel('Year')
plt.ylabel('Total FRP')
plt.title('Total Fire Radiative Power (FRP) by Year')
plt.grid(True)
plt.show()


In [ ]:
# Count the number of observations per year
yearly_counts = df.groupby('year').size().reset_index(name='count')

# Plot the count of observations per year
plt.figure(figsize=(5, 3))
plt.bar(yearly_counts['year'], yearly_counts['count'], color='lightgreen')
plt.xlabel('Year')
plt.ylabel('Number of Observations')
plt.title('Number of Observations per Year')
plt.grid(True)
plt.show()
